In [1]:
#!pip install chesslab

In [2]:
import numpy as np
from chesslab.utils import load_pkl
from chesslab.training_tf import fitting
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:

lr = 0.1
epochs=20
batch_size = 128
test_percent=0.1

path = 'D:/database/ccrl/'
name_data='ccrl_states_elo3.pkl'
name_labels='ccrl_results_elo3.pkl'
save_name='./tmp/tf_weights-relu-elo3.1'

optim = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [4]:
download=False
if download:
    from chesslab.utils import download_7z
    path='./'
    file_id = '1MFHFz_rxNziYSeN-9ruwnRiYskd0_9ss'
    download_7z(file_id,path)

In [5]:
encoding_3={
    '.':np.array([0,0,0,0,0,0,0,0,0,0,0,0],dtype=np.float32),
    'p':np.array([1,0,0,0,0,0,0,0,0,0,0,0],dtype=np.float32),
    'P':np.array([0,0,0,0,0,0,0,0,0,0,0,1],dtype=np.float32),
    'b':np.array([0,1,0,0,0,0,0,0,0,0,0,0],dtype=np.float32),
    'B':np.array([0,0,0,0,0,0,0,0,0,0,1,0],dtype=np.float32),
    'n':np.array([0,0,1,0,0,0,0,0,0,0,0,0],dtype=np.float32),
    'N':np.array([0,0,0,0,0,0,0,0,0,1,0,0],dtype=np.float32),
    'r':np.array([0,0,0,1,0,0,0,0,0,0,0,0],dtype=np.float32),
    'R':np.array([0,0,0,0,0,0,0,0,1,0,0,0],dtype=np.float32),
    'q':np.array([0,0,0,0,1,0,0,0,0,0,0,0],dtype=np.float32),
    'Q':np.array([0,0,0,0,0,0,0,1,0,0,0,0],dtype=np.float32),
    'k':np.array([0,0,0,0,0,1,0,0,0,0,0,0],dtype=np.float32),
    'K':np.array([0,0,0,0,0,0,1,0,0,0,0,0],dtype=np.float32)
}


In [6]:
class Model_3():

    def __init__(self,
                   n_classes=2):
        initializer = tf.keras.initializers.GlorotNormal()
        self.hw=[]
        self.hb=[]

        self.hw.append( tf.Variable(initializer(shape=(7,7,12,32),dtype=np.float32),name="hl1weigths",dtype="float32") )
        self.hb.append( tf.Variable(np.zeros(32,dtype=np.float32),name="hl1bias",dtype="float32") )
        #8x8x32
        self.hw.append( tf.Variable(initializer(shape=(5,5,32,64),dtype=np.float32),name="hl2weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(64,dtype=np.float32),name="hl2bias",dtype="float32"))
        #8x8x64
        self.hw.append( tf.Variable(initializer(shape=(3,3,64,128),dtype=np.float32),name="hl3weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(128,dtype=np.float32),name="hl3bias",dtype="float32"))
        #8x8x128
        self.hw.append( tf.Variable(initializer(shape=(8*8*128,256),dtype=np.float32),name="hl4weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(256,dtype=np.float32),name="hl4bias",dtype="float32"))

        self.hw.append( tf.Variable(initializer(shape=(256, n_classes),dtype=np.float32),name="outweigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(n_classes,dtype=np.float32),name="outbias",dtype="float32"))

        self.trainable_variables = []
        for i in range(len(self.hw)):
            self.trainable_variables.append(self.hw[i])    
            self.trainable_variables.append(self.hb[i])

    def __call__(self,x): 

        out = tf.cast(x, tf.float32)
        out = tf.reshape(out, shape=[-1, 8, 8, 12])

        layer=0
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*32
        layer+=1
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*64
        layer+=1  
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*128
        layer+=1
        out =  tf.reshape(out,[-1, 8*8*128])
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)

        layer+=1
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])

        return out

In [7]:
np.random.seed(0)
tf.random.set_seed(0)

x_data = load_pkl(path+name_data)
y_data = load_pkl(path+name_labels)[:,1] #Nota: pasa de onehot a logits

print(x_data.shape)
print(y_data.shape)

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size = test_percent, random_state = 0, shuffle = True)

del x_data
del y_data

(2660430, 64)
(2660430,)


In [8]:

model = Model_3()
encoding=encoding_3


In [9]:
fitting(epochs=epochs,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model=model,
        optimizer=optim,
        batch_size=batch_size,
        lr=lr,
        loss_fn=loss_fn,
        save_name=save_name,
        encoding=encoding)

2021-10-29 16:34:20


Epoch: 01/20 | time: 100s = 1.7m | train loss: 0.5088 | train acc: 0.7430 | test loss: 0.4867 | test acc: 0.7573

Epoch: 02/20 | time: 98s = 1.6m | train loss: 0.3760 | train acc: 0.8261 | test loss: 0.3707 | test acc: 0.8332

Epoch: 03/20 | time: 96s = 1.6m | train loss: 0.2480 | train acc: 0.8929 | test loss: 0.2315 | test acc: 0.9028

Epoch: 04/20 | time: 107s = 1.8m | train loss: 0.1666 | train acc: 0.9305 | test loss: 0.1772 | test acc: 0.9275

...

Epoch: 16/20 | time: 105s = 1.7m | train loss: 0.0299 | train acc: 0.9887 | test loss: 0.1164 | test acc: 0.9671

Epoch: 17/20 | time: 104s = 1.7m | train loss: 0.0280 | train acc: 0.9895 | test loss: 0.1264 | test acc: 0.9658

Epoch: 18/20 | time: 104s = 1.7m | train loss: 0.0273 | train acc: 0.9899 | test loss: 0.1196 | test acc: 0.9681

Epoch: 19/20 | time: 103s = 1.7m | train loss: 0.0264 | train acc: 0.9903 | test loss: 0.1165 | test acc: 0.9683

Epoch: 20/20 | time: 103s = 1.7m | train loss: 0.0252 | train acc: 0.9908 | test loss: 0.1321 | test acc: 0.9668

In [10]:
fitting(epochs=10,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model= model, 
        load_name='tmp/tf_weights-relu-elo3.1.20.h5',
        save_name=save_name,)

2021-10-29 17:08:55


Epoch: 21/30 | time: 103s = 1.7m | train loss: 0.0247 | train acc: 0.9911 | test loss: 0.1229 | test acc: 0.9687

Epoch: 22/30 | time: 104s = 1.7m | train loss: 0.0238 | train acc: 0.9914 | test loss: 0.1189 | test acc: 0.9704

Epoch: 23/30 | time: 103s = 1.7m | train loss: 0.0234 | train acc: 0.9916 | test loss: 0.1189 | test acc: 0.9703

Epoch: 24/30 | time: 104s = 1.7m | train loss: 0.0231 | train acc: 0.9918 | test loss: 0.1290 | test acc: 0.9688

...

Epoch: 26/30 | time: 103s = 1.7m | train loss: 0.0222 | train acc: 0.9922 | test loss: 0.1266 | test acc: 0.9701

Epoch: 27/30 | time: 104s = 1.7m | train loss: 0.0224 | train acc: 0.9922 | test loss: 0.1305 | test acc: 0.9696

Epoch: 28/30 | time: 104s = 1.7m | train loss: 0.0225 | train acc: 0.9922 | test loss: 0.1265 | test acc: 0.9705

Epoch: 29/30 | time: 103s = 1.7m | train loss: 0.0223 | train acc: 0.9924 | test loss: 0.2591 | test acc: 0.9509

Epoch: 30/30 | time: 104s = 1.7m | train loss: 0.0224 | train acc: 0.9924 | test loss: 0.1228 | test acc: 0.9713